**DATASET**
: Let's use this dataset for training on Kaggle: [/kaggle/input/airbus-ship-detection](https://www.kaggle.com/c/airbus-ship-detection)

In [1]:
H = 768
WORK_DIR = "/kaggle/working/"

# **Step 1: Pre-processing data for YOLO Format**

In [2]:
import pandas as pd
import numpy as np
def encode_to_bounding_box(encode_value, check_test = False):
    """Function help convert Encode Pixel value to Bounding Box for YoLo Training for a picture"""
    encode_value = list(map(int, encode_value.split()))
    x_min, y_min, x_max, y_max = 1000, 1000, -1, -1
    for i in range(0, len(encode_value), 2):
        start = encode_value[i]
        for j in range(start, start + encode_value[i+1]):
            r = (j-1) % H
            c = (j-1) // H
            if x_min > c: x_min = c
            if y_min > r: y_min = r
            if x_max < c: x_max = c
            if y_max < r: y_max = r
    if x_min > 0 and y_min > 0: 
        x_min, y_min = x_min - 1, y_min - 1
    if x_max > 0 and y_max > 0:
        x_max, y_max = x_max + 1, y_max + 1
    if check_test == True:
        return [x_min, y_min, x_max, y_max]
    x_center = (x_min + x_max) / 2 / H
    y_center = (y_min + y_max) / 2 / H
    w = (x_max - x_min) / H
    h = (y_max - y_min) / H
    bb = [0, x_center, y_center, w, h]
    return " ".join(map(str, bb))


In [3]:
import os
import shutil

images_path = os.path.join(WORK_DIR, "images")
img_train = os.path.join(images_path, "train")
img_val = os.path.join(images_path, "val")
img_test = os.path.join(images_path, "test")

labels_path = os.path.join(WORK_DIR, "labels")
lb_train = os.path.join(labels_path, "train")
lb_val = os.path.join(labels_path, "val")
lb_test = os.path.join(labels_path, "test")

if os.path.exists(images_path) and os.path.exists(labels_path):
    shutil.rmtree(images_path)
    shutil.rmtree(labels_path)
for path in [images_path, img_train, img_val, img_test,
         labels_path, lb_train, lb_val, lb_test]:
    os.makedirs(path, exist_ok=True)

In [ ]:
train_list = pd.read_csv("/kaggle/input/airbus-ship-detection/train_ship_segmentations_v2.csv")
has_ship_train = 90000
has_ship_val = 10000

index_seg = 0
def setup_dataset(has_ship_len, index_seg, img_path, lb_path, tag = 'train'):
    has = 0
    multiple = False
    while has < has_ship_len:
        if pd.isna(train_list.iloc[index_seg,1]) == False:
            img_name = train_list.iloc[index_seg,0]
            
            src_img_path = "/kaggle/input/airbus-ship-detection/train_v2/"+img_name
            full_img_path = os.path.join(img_path, img_name)
            shutil.copy(src_img_path, full_img_path) #copy ảnh vào thư mục cần 
            
            full_lb_path = os.path.join(lb_path, img_name.replace(".jpg", ".txt"))
            bbs = []
            while train_list.iloc[index_seg,0] == img_name:
                multiple = True
                bbs.append(encode_to_bounding_box(train_list.iloc[index_seg,1]))
                index_seg += 1
            with open(full_lb_path, "w") as f:
                for item in bbs:
                    f.write(item + "\n")
        has += 1
        if has % 2000 == 0: print("Processed ",tag,": ",has)
        if multiple == False: index_seg += 1
        else: multiple = False
    print("Check index_seg:", index_seg)
    return index_seg

index_seg = setup_dataset(has_ship_train, index_seg, img_train, lb_train, 'train') #SETUP TRAIN
index_seg = setup_dataset(has_ship_val, index_seg, img_val, lb_val, 'validation') #SETUP VALIDATION
#setup_dataset(has_ship_test, index_seg, img_test, lb_test, 'test') #SETUP TEST

Processed  train :  2000
Processed  train :  4000
Processed  train :  6000
Processed  train :  8000
Processed  train :  10000
Processed  train :  12000
Processed  train :  14000
Processed  train :  16000
Processed  train :  18000
Processed  train :  20000
Check index_seg: 23930
Processed  validation :  2000
Processed  validation :  4000
Processed  validation :  6000
Processed  validation :  8000
Processed  validation :  10000
Check index_seg: 35972


# **Step 2: YOLO Setup**

In [12]:
# !pip install ultralytics

In [ ]:
import yaml
import os
import torch
from ultralytics import YOLO

# Tạo file dataset.yaml
yaml_file = '/kaggle/working/dataset.yaml'
if os.path.exists(yaml_file):
    os.remove(yaml_file)
    
dataset_info = {
    'path': '/kaggle/working',
    'train': 'images/train',
    'val': 'images/val', 
    'test': 'images/test',
    'nc': 1,                      
    'names': ['ship'] 
}

with open(yaml_file, 'w') as f:
    yaml.dump(dataset_info, f, default_flow_style=False)


Đã tạo dataset.yaml


# **Step 3: YOLO Quantization-aware Training**

In [ ]:
import torch
from ultralytics import YOLO
from ultralytics.nn.modules import Conv
from torch import nn
import torch.quantization as tq

In [ ]:
model = YOLO("yolov8n.pt")

In [ ]:
def inspect_yolo_structure(model, max_depth=3):
    """Inspect YOLO model structure để hiểu cách fuse"""
    def print_structure(module, prefix="", depth=0):
        if depth > max_depth:
            return
        
        for name, child in module.named_children():
            full_name = f"{prefix}.{name}" if prefix else name
            print(f"{'  ' * depth}{full_name}: {type(child).__name__}")
            
            # Print attributes nếu có conv, bn, act
            attrs = []
            if hasattr(child, 'conv'):
                attrs.append('conv')
            if hasattr(child, 'bn'): 
                attrs.append('bn')
            if hasattr(child, 'act'):
                attrs.append('act')
            if attrs:
                print(f"{'  ' * depth}  -> has: {attrs}")
            
            print_structure(child, full_name, depth + 1)
    print_structure(model)
#inspect_yolo_structure(model)

In [ ]:
def fuse_conv_bn_only(model):
    """
    Fuse: Conv + BatchNorm, Skip SiLU activation
    """
    print("=== Fusing Conv+BN Only (SiLU Preserved) ===")
    
    fused_count = 0
    skipped_count = 0
    
    def fuse_recursive(module, prefix=""):
        nonlocal fused_count, skipped_count
        
        for name, child in module.named_children():
            full_name = f"{prefix}.{name}" if prefix else name
            
            # Target: YOLO Conv modules (có conv, bn, act)
            if isinstance(child, Conv):
                if hasattr(child, 'conv') and hasattr(child, 'bn'):
                    try:
                        # Fuse CHÍNH XÁC conv + bn, BỎ QUA act
                        torch.quantization.fuse_modules(
                            child, 
                            ['conv', 'bn'],  # CHỈ 2 modules này
                            inplace=True
                        )
                        fused_count += 1
                        act_type = type(child.act).__name__ if hasattr(child, 'act') else 'None'
                        #print(f"✅ Fused Conv+BN: {full_name} (Act preserved: {act_type})")
                    except Exception as e:
                        skipped_count += 1
                        #print(f"⚠️  Skip {full_name}: {e}")
            
            # Handle generic modules chỉ có conv + bn
            elif hasattr(child, 'conv') and hasattr(child, 'bn'):
                if isinstance(child.conv, nn.Conv2d) and isinstance(child.bn, nn.BatchNorm2d):
                    try:
                        torch.quantization.fuse_modules(child, ['conv', 'bn'], inplace=True)
                        fused_count += 1
                        act_type = type(child.act).__name__ if hasattr(child, 'act') else 'None'
                        #print(f"✅ Fused Conv+BN: {full_name} (Act preserved: {act_type})")
                    except Exception as e:
                        skipped_count += 1
                        #print(f"⚠️  Skip {full_name}: {e}")
            
            # Recursive để đi sâu vào tất cả modules
            fuse_recursive(child, full_name)
    
    fuse_recursive(model)
    
    print(f"\n📊 Fusion Summary:")
    print(f"  ✅ Successfully fused: {fused_count} Conv+BN pairs")
    print(f"  ⚠️  Skipped: {skipped_count} modules")
    print(f"  🎯 Strategy: Conv+BN fusion only, SiLU preserved")
    
    return model

yolo_torch_model = model.model
fused_torch_model = fuse_conv_bn_only(yolo_torch_model)

In [ ]:
def verify_conv_bn_fusion(model):
    """
    Verify rằng Conv+BN đã được fuse, SiLU vẫn còn
    """
    print("\n=== Verifying Conv+BN Fusion ===")
    
    conv_bn_count = 0
    silu_count = 0
    other_act_count = 0
    
    def count_modules(module, prefix=""):
        nonlocal conv_bn_count, silu_count, other_act_count
        
        for name, child in module.named_children():
            full_name = f"{prefix}.{name}" if prefix else name
            
            # Count fused Conv+BN (should have ConvBn2d after fusion)
            if 'ConvBn2d' in str(type(child)) or 'ConvBnReLU2d' in str(type(child)):
                conv_bn_count += 1
                #print(f"🔗 Fused ConvBN: {full_name}")
            
            # Count SiLU activations
            elif isinstance(child, nn.SiLU):
                silu_count += 1
                #print(f"⚡ SiLU preserved: {full_name}")
            
            # Count other activations
            elif isinstance(child, (nn.ReLU, nn.ReLU6, nn.GELU, nn.Hardswish)):
                other_act_count += 1
                #print(f"🔥 Other activation: {full_name} ({type(child).__name__})")
            
            count_modules(child, full_name)
    
    count_modules(model)
    
    print(f"\n📊 Verification Results:")
    print(f"  🔗 Fused Conv+BN modules: {conv_bn_count}")
    print(f"  ⚡ Preserved SiLU: {silu_count}")  
    print(f"  🔥 Other activations: {other_act_count}")
    
    if conv_bn_count > 0:
        print("✅ Conv+BN fusion successful!")
    else:
        print("❌ No fused Conv+BN found!")
        
    if silu_count > 0:
        print("✅ SiLU activations preserved!")
    else:
        print("⚠️  No SiLU found (might be normal)")

verify_conv_bn_fusion(fused_torch_model)

In [ ]:
def setup_qat_config_for_silu():
    """
    QAT config tối ưu cho model có SiLU không được fuse
    """
    print("Setting up QAT config for SiLU preservation...")
    
    # Sử dụng symmetric quantization cho SiLU (có negative values)
    from torch.quantization.fake_quantize import FusedMovingAvgObsFakeQuantize
    from torch.quantization.observer import MovingAverageMinMaxObserver
    
    # Activation fake quantizer - symmetric cho SiLU
    activation_fake_quant = FusedMovingAvgObsFakeQuantize.with_args(
        observer=MovingAverageMinMaxObserver,
        quant_min=-128,
        quant_max=127,
        dtype=torch.qint8,
        qscheme=torch.per_tensor_symmetric,
        reduce_range=False
    )
    
    # Weight fake quantizer - per channel cho better accuracy
    weight_fake_quant = tq.default_weight_fake_quant
    
    qconfig = tq.QConfig(
        activation=activation_fake_quant,
        weight=weight_fake_quant
    )
    
    print("✅ QAT config created for SiLU compatibility")
    return qconfig

qat_config = setup_qat_config_for_silu()
fused_torch_model.qconfig  = qat_config
fused_torch_model.train()
tq.prepare_qat(fused_torch_model, inplace=True)
yolo_wrapper = model

yolo_wrapper.model = fused_torch_model



try:
    yolo_wrapper.train(data='/kaggle/working/dataset.yaml', imgsz=768, epochs=99, batch=32, project='yolov11qat_ship', name='train_run', exist_ok=True)
    
except Exception as e:
    print(f"❌ Training failed: {e}")
    import traceback
    traceback.print_exc()

# **Post-processing for INT8 Quantization  & Evaluation**

In [ ]:
!cp /kaggle/input/8nqat/pytorch/default/1/8nqat.pt /kaggle/working/8nqat.pt

In [ ]:
model_path = "/kaggle/working/8nqat.pt"

In [ ]:
from ultralytics import YOLO

# Load model
model = YOLO(model_path)

# Export sang OpenVINO INT8
model.export(format="openvino", int8=True)

In [ ]:
int8_model = YOLO("/kaggle/working/11fpn_int8_openvino_model")

# Validate
results = int8_model.val(data="/kaggle/working/dataset.yaml", imgsz=768, split="val")
print(results)

In [ ]:
# Lấy metrics dictionary trực tiếp
metrics = results.results_dict  # KHÔNG có ()

# In ra các chỉ số
print("mAP@0.5:", metrics["metrics/mAP50(B)"])
print("mAP@0.5:0.95:", metrics["metrics/mAP50-95(B)"])
print("Precision:", metrics["metrics/precision(B)"])
print("Recall:", metrics["metrics/recall(B)"])